In [1]:
from src.v1.symbolic import Par, Var
from src.v1.mdaobuild import optsolver, solver, explicit
import openmdao.api as om
import sympy as sp

### OpenMDAO run 1

In [2]:
a,x = Var('a'), Var('x')
eqs = {
    0: (a, x**2-2*x),
    1: (None, a+sp.sin(x)-1),
    2: (None, -x+2)
}

In [3]:
prob = om.Problem()
model = prob.model
groups = {0: model}
solver(groups, eqs, 0, 1, [x], [1]) 
explicit(groups, eqs, 1, a, 0)
prob.setup();
prob.set_solver_print(level=1)
prob.run_model()


group1
NL: Newton Converged in 6 iterations


In [4]:
model.list_outputs();

1 Explicit Output(s) in 'model'

varname  val         
-------  ------------
group1
  eq0
    a    [0.11550053]


1 Implicit Output(s) in 'model'

varname  val         
-------  ------------
group1
  res1
    x    [2.05617258]




## OpenMDAO run 2

In [5]:
p = Par('p', 0.1)
a,b,c,d,e,x = Var('a'), Var('b'), Var('c'), Var('d'), Var('e'), Var('x')
eqsn = {
    0: (a, 2.0*x),
    1: (x, b - 0.5),
    2: (b, 2.0*c - 0.2*d - 40.0),
    3: (c, d + 10),
    4: (d, -0.5*e)
}

In [6]:
prob = om.Problem()
model = prob.model
groups = {0:model}

In [7]:
explicit(groups, eqsn, 0, d, 4)
explicit(groups, eqsn, 0, c, 3)
explicit(groups, eqsn, 0, b, 2)
explicit(groups, eqsn, 0, x, 1)
explicit(groups, eqsn, 0, a, 0)

In [8]:
optsolver(groups, {5: (None, a**2+d**2+b**2)}, 
          0, 1, [e], [[],[],[5]])

In [9]:
prob.setup();

In [10]:
#prob.set_val('c', 100)
prob.set_solver_print(level=1)
prob.driver = om.ScipyOptimizeDriver()
prob.run_driver();

Optimization terminated successfully    (Exit mode 0)
            Current function value: 121.17674418604648
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
Optimization Complete
-----------------------------------


In [11]:
model.list_inputs();

9 Input(s) in 'model'

varname  val           
-------  --------------
eq4
  e      [-21.34883721]
eq3
  d      [10.6744186]  
eq2
  d      [10.6744186]  
  c      [20.6744186]  
eq1
  b      [-0.78604651] 
eq0
  x      [-1.28604651] 
eq5
  a      [-2.57209302] 
  d      [10.6744186]  
  b      [-0.78604651] 




In [12]:
prob.set_val('e', -21.35)
prob.run_model()
prob.get_val('f05'), prob.get_val('e')

(array([121.17675]), array([-21.35]))

## OpenMDAO run 3

In [13]:
from src.v1.symbolic import Par, Var
from src.v1.mdaobuild import optsolver, solver, explicit
import openmdao.api as om

In [15]:
p = Par('p', 0.1)
a,b,c,d,e,x = Var('a'), Var('b'), Var('c'), Var('d'), Var('e'), Var('x')

eqs = {
    0: (x, 0.5*a),
    1: (b, x + 0.5),
    2: (c, p*d+20+0.5*b),
    3: (d, c - 10),
    4: (None, d + 0.5*e)
}

In [32]:
prob = om.Problem()
model = prob.model
groups = {0:model}

In [33]:
solver(groups, eqs, 0, 1, [a], [4])
explicit(groups, eqs, 1, x, 0)
explicit(groups, eqs, 1, b, 1)
solver(groups, eqs, 1, 2, [c,d], [2,3])
optsolver(groups, {5: (None, a**2+d**2+b**2)}, 0, 1, [e], [[],[],[5]])

In [34]:
prob.setup();

In [35]:
prob.set_val('e', -21.35)
prob.set_val('p', 0.1)
prob.run_model()


group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 1 iterations
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations


In [36]:
prob.set_solver_print(level=1)
prob.driver = om.ScipyOptimizeDriver()
prob.run_driver();


group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 0 iterations

group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 0 iterations

group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

group1
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
+  
+  =============
+  group1.group2
+  =============
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: 121.17674418604649
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
Optimization Complete
---------------------

In [37]:
model.list_inputs();

9 Input(s) in 'model'

varname    val           
---------  --------------
group1
  res4
    e      [-21.34883721]
    d      [10.6744186]  
  eq0
    a      [-2.57209302] 
  eq1
    x      [-1.28604651] 
  group2
    res23
      b    [-0.78604651] 
      p    [0.1]         
eq5
  a        [-2.57209302] 
  d        [10.6744186]  
  b        [-0.78604651] 


